In [ ]:
from pathlib import Path

import geopandas as gpd
import numpy as np
import osmnx as ox
import pandas as pd
from common import load_crs, load_stops
from shapely import Point, Polygon

ox.settings.use_cache = True
ox.settings.log_console = False

crs = load_crs()

In [ ]:
def read_boundary(city: str) -> Polygon:
    boundary = gpd.read_file(f"../output/{city}/boundary.geojson").set_crs(4326)
    return boundary.geometry[0]


def determine_city_centroid_by_landuse(
    boundary: Polygon,
    crs: int,
    landuse: list[str] = ["residential"],
    # landuse: list[str] = ["residential", "retail", "industrial"],
) -> Point:
    landuse = ox.features_from_polygon(
        boundary,
        tags={"landuse": landuse},
    )
    landuse = landuse[landuse["geometry"].geom_type == "Polygon"].copy()
    landuse.to_crs(crs, inplace=True)
    return landuse.union_all().centroid


# based on: https://stackoverflow.com/a/38022636/4737417
def distance_matrix(data: pd.DataFrame):
    result = []
    for i in data.itertuples():
        r = []
        for j in data.itertuples():
            d = np.round(i.geometry.distance(j.geometry))
            r.append(d)
        result.append(r)
    result = np.array(result)
    return result


def medoid_index(distamce_matrix: np.array) -> int:
    return int(np.argmin(distamce_matrix.sum(axis=0)))


def calculate_medoid(data: pd.DataFrame, id_column: str = "stop_id") -> int:
    dmx = distance_matrix(data)
    i = int(np.argmin(dmx.sum(axis=0)))
    return data[id_column].tolist()[i]

In [2]:
CITY = "helsinki"
# possible values: Eigenvector Centrality, Degree Centrality, Closeness Centrality, Betweenness Centrality
CENTRALITY = "Betweenness Centrality"

In [ ]:
stops = load_stops(CITY)
stops.to_crs(crs[CITY], inplace=True)

In [ ]:
landuse_centroid = determine_city_centroid_by_landuse(read_boundary(CITY), crs[CITY])

In [7]:
maxc = stops[CENTRALITY].max()
medoid_id = calculate_medoid(stops[stops["Betweenness Centrality"] == maxc])
medoid = stops[stops["stop_id"] == medoid_id]["geometry"].tolist()[0]
centroid = stops[stops["Betweenness Centrality"] == maxc].union_all().centroid

In [8]:
stops["distance_from_largest_betweenness_medoid"] = stops["geometry"].apply(
    lambda x: np.round(x.distance(medoid) / 1000, 3)
)
stops["distance_from_largest_betweenness_centroid"] = stops["geometry"].apply(
    lambda x: np.round(x.distance(centroid) / 1000, 3)
)
stops["distance_from_landuse_centroid"] = stops["geometry"].apply(
    lambda x: np.round(x.distance(landuse_centroid) / 1000, 3)
)

In [ ]:
distance = stops[
    [
        "stop_id",
        "distance_from_largest_betweenness_centroid",
        "distance_from_largest_betweenness_medoid",
        "distance_from_landuse_centroid",
    ]
].copy()
Path(f"../output/{CITY}").mkdir(parents=True, exist_ok=True)
distance.dropna(subset=["stop_id"]).to_csv(
    f"../output/{CITY}/distance.csv", index=False
)